In [1]:
import numpy as np
from tensorflow import keras
from keras import models
from keras import layers
from keras.datasets import imdb

2021-08-06 15:37:58.018329: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-06 15:37:58.018425: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:

(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.9/site-packages/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.9/site-packages/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object'

In [3]:
training_data.shape, training_targets.shape, testing_data.shape, testing_targets.shape, data.shape, targets.shape

((25000,), (25000,), (25000,), (25000,), (50000,), (50000,))

In [4]:
data = data[0:1000, ]
targets = targets[0:1000, ]

In [5]:
def stats(targets, data):
    print("Train Categories:", np.unique(targets))
    print("Number of unique words:", len(np.unique(np.hstack(data))))

    length = [len(i) for i in data]
    print("Average Review length:", np.mean(length))
    print("Longest Review length:", np.max(length))
    print("Shortest Review length:", np.min(length))
    print("Standard Deviation:", round(np.std(length)))

stats(training_targets, training_data)

Train Categories: [0 1]
Number of unique words: 9998
Average Review length: 238.71364
Longest Review length: 2494
Shortest Review length: 11
Standard Deviation: 176


In [6]:
stats(testing_targets, testing_data)

Train Categories: [0 1]
Number of unique words: 9951
Average Review length: 230.8042
Longest Review length: 2315
Shortest Review length: 7
Standard Deviation: 169


In [43]:
max(reverse_index.keys())

88584

In [2]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in training_data[0]] )
print(decoded) 

1654784/1641221 [==============================] - 0s 0us/step


NameError: name 'training_data' is not defined

# Data Prep

In [3]:
def w_to_index(text, index):
    lst = []
    non_exist = 9999999
    for w in text.split(" "):
        try:
            w_indx = index[w.lower()]
        except:
            w_indx = non_exist
        lst += [w_indx]
    return([lst])

In [4]:
def vectorize(sequences, dimension = 10000):
    non_exist = 9999999
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        sequence = [w for w in sequence if w != non_exist]
        results[i, sequence] = 1
    return results
 
#training_data = vectorize(training_data)
#testing_data = vectorize(testing_data)
#data = vectorize(data)
#targets = np.array(targets).astype("float32")

In [17]:
test_x = data[:900]
test_y = targets[:900]
train_x = data[100:]
train_y = targets[100:]

In [21]:
#test_x = vectorize(test_x)

In [22]:
test_x.shape, train_x.shape

((900, 10000), (900,))

In [15]:
text = 'I like the movie'
def text_vectorize(text):
    text_index = w_to_index(text, index)
    vect = vectorize(text_index, dimension = 10000)
    return(vect)
text_vectorize(text)

array([[0., 0., 0., ..., 0., 0., 0.]])

In [40]:
text = 'I like as movie apple ghgas jlkgf'
np.sum(text_vectorize(text))

5.0

# Model Training

In [34]:
# model = keras.Sequential()
# # Input - Layer
# model.add()
# # Hidden - Layers
# model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
# model.add(layers.Dense(50, activation = "relu"))
# model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
# model.add(layers.Dense(50, activation = "relu"))
# # Output- Layer
# model.add(layers.Dense(1, activation = "sigmoid"))

model = keras.models.Sequential([
    layers.Dense(50, activation = "relu", input_shape=(10000, )),
    layers.Dropout(0.3, noise_shape=None, seed=None),
    layers.Dense(50, activation = "relu"),
    layers.Dropout(0.2, noise_shape=None, seed=None),
    layers.Dense(50, activation = "relu"),
    layers.Dense(1, activation = "sigmoid")
    
])

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [35]:
results = model.fit(
 train_x, train_y,
 epochs= 2,
 batch_size = 500,
 validation_data = (test_x, test_y)
)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [23]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_24 (ModuleWra (None, 50)                500050    
_________________________________________________________________
module_wrapper_25 (ModuleWra (None, 50)                0         
_________________________________________________________________
module_wrapper_26 (ModuleWra (None, 50)                2550      
_________________________________________________________________
module_wrapper_27 (ModuleWra (None, 50)                0         
_________________________________________________________________
module_wrapper_28 (ModuleWra (None, 50)                2550      
_________________________________________________________________
module_wrapper_29 (ModuleWra (None, 1)                 51        
Total params: 505,201
Trainable params: 505,201
Non-trainable params: 0
________________________________________________

In [22]:
print(np.mean(results.history["val_accuracy"]))


0.7561111152172089


# save model

In [24]:
model.save('my_model')

2021-08-05 02:33:39.695625: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [6]:
new_model = keras.models.load_model('my_model')

2021-08-06 01:22:06.732273: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-06 01:22:06.732520: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-06 01:22:06.732602: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (26b9a757ed92): /proc/driver/nvidia/version does not exist
2021-08-06 01:22:06.733598: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
loss, acc = new_model.evaluate(test_x, test_y, verbose=2)
acc

29/29 - 0s - loss: 0.6649 - accuracy: 0.8056


0.8055555820465088

# Monitoring

In [1]:
import wandb
experiment_name = wandb.util.generate_id()

run=wandb.init(project='IMDB sentiment',
           group=experiment_name,
           notes='This is Test Run')


wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
2021-08-05 11:41:58.763920: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-05 11:41:58.763993: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [41]:
def monitor(text, text_vec):
    text_len = len(text.split(" "))
    unique_words_perct = round(len(set(text.split(" ")))/text_len * 100, 1)
    new_text = np.sum(text_vec)
    new_text_perct = np.sum(text_vec)/text_len
    return(text_len, unique_words_perct, new_text, new_text_perct)

In [42]:
def pred(model, text):
    time0 = time.time()
    text_vec = text_vectorize(text)
    pred = model(text_vec)
    pred_time = time.time() - time0
    text_len, unique_words_perct, new_text, new_text_perct = monitor(text, text_vec)
    return(pred, pred_time, text_len, unique_words_perct, new_text, new_text_perct)

In [49]:
import time
text = "I hate the movie tda"
pred, pred_time, text_len, unique_words_perct, new_text, new_text_perct = pred(new_model, text)

In [50]:
wandb.log({
       "Prediction Time": pred_time,
       "Input Text Word Count": text_len,
       "Input Text Unique Word Perct": unique_words_perct,
       "Input Text Unseen Word Count": new_text,
       "Input Text Unseen Word Perct": new_text_perct})